<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de otra playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 2 - Obtención de datos adicionales</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Creación de etiquetas](#section2)
    * [2.1. - Construcción de la matriz TF-IDF](#section21)
* [3. Obtención de géneros musicales (artistas)](#section3)
* [4. Obtención de fechas de lanzamiento (álbumes)](#section4)
* [5. Obtención de características músicales (pistas)](#section5)

<br>

---

In [2]:
import os
import pandas as pd
import re

from collections import defaultdict
from modules import emojis
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [3]:
MPD_CSV_PATH = 'MPD_CSV'
TFIDF_DATA_FILE = os.path.join(MPD_CSV_PATH,'mpd.tfidf-data.pkl')

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Con el fin de obtener mejores predicciones, vamos a obtener información adicional sobre las pistas que conforman una playlist empleando [*Spotipy*](https://spotipy.readthedocs.io) para acceder a la [API de *Spotify*](https://developer.spotify.com/documentation/web-api/). Mediante uso de técnicas de *Procesamiento del Lenguaje Natural* o *NLP* (*Natural Language Processing*), también vamos a procesar los títulos de las playlist para convertirlos en etiquetas o *tokens*, con lo cual conseguiremos relacionar playlists de títulos similares.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section2"></a>
## <font color="#92002A">2 - Creación de etiquetas</font>
<br>

Para la creación de etiquetas, con las que identificaremos las playlists y nos ayudaran a relacionarlas con otras similares, partiremos de los títulos y los procesaremos de tal manera que queden en un formato común (todos los caracteres en minúscula, sin signos de puntuación, ...). También identificaremos aquellas playlists cuyo nombre contiene algún emoticono y transformarlo a texto, todo ello obteniendo los términos más frecuentes que aparecen junto a cada emoticono.

<br>

Primero, creamos un *dataframe* en el que almacenaremos los títulos de las playlist, y los *emojis* que contiene cada playlist:

In [4]:
df_names = pd.read_csv('MPD_CSV/mpd.playlists-info.csv')['name'].to_frame()
df_names = pd.concat([df_names, pd.read_csv('MPD_CSV/mpd.playlists-info-test.csv', index_col=0)['name'].to_frame()])

df_names['name'] = df_names['name'].astype(str)
df_names.index.name = "pl_pid"

In [5]:
df_names['emojis'] = df_names['name'].apply(emojis.get_emojis_string)

<br>

Una vez que se ha creado el *dataframe* con los títulos de las playlists y los emojis que posee, vamos a crear una serie de funciones con las que eliminaremos del texto aquellos elementos que no deseamos:

* `replace_apostrophe`: Se encarga de transformar algunos casos particulares donde el apóstrofe aparece en el título.
* `dot_remover`: Elimina el punto de los títulos y cuando una palabra está separada por puntos (por ejemplo: H.A.P.P.Y. o 1.9.9.9.), elimina dichos puntos y junta los caracteres para formar la palabra. Como excepción si el título contiene un número, lo mantendremos.
* `clean_name`: Función que se encarga de eliminar caracteres no deseados y transformar el texto para que todos los títulos tengan un formato común.

In [6]:
def replace_apostrophe(text):
    # Como caso excepcional, vamos a hacer que el género R'n'B  se conserve como
    # una única palabra, rnb.
    if text.lower() == "r'n'b":
        return "rnb"
    
    # Adaptamos décadas como 1990's ó 90's para que aparezcan como
    # 1990s ó 90s.
    text = re.sub(r"([0-9]+)'s", r'\1s', text)
    text = text.replace("'s ", " ")
    
    text = re.sub(r"([0-9]+)´s", r'\1s', text)
    text = text.replace("´s ", " ")
    
    return text

In [7]:
def dot_remover(text):
    # Si el texto contiene un punto, realiza la comprobación.
    if "." in text:
        d = dict()
        
        for token in word_tokenize(text):
            cond_1 = "." in token
            cond_2 = len(token) > 2
            cond_3 = token.count('.') >= int(len(token)/2)
            cond_4 = not(token.count('.') == 1 and token.replace(".","").isnumeric())
            # Comprobamos si la palabra es separada por puntos.
            if all([cond_1, cond_2, cond_3, cond_4]):
                d[token] = token.replace('.','')
            else:
                # Comprobamos si la palabra del texto es un número.
                result = re.match(r"^[0-9]+([,.][0-9]+)?$", token)
                if result != None:
                    d[result.string] = result.string.replace(",","{dot}").replace(".","{dot}")

        for k,v in d.items():
            text = text.replace(k,v)
        
        text = text.replace("."," ")
        text = text.replace("{dot}",".")
        
    return text

In [8]:
def clean_name(name):
    name = str(name).lower() # Convierte todo a minúscula.
    name = replace_apostrophe(name)
    name = dot_remover(name)
    name = re.sub(r"[~_]", ' ', name) # Elimina los caracteres '~' y '_'.
    name = re.sub(r"[^\w\s\.]", ' ', name) # Elimina signos de puntuación.
    
    tokens = list()
    for token in word_tokenize(name):
        token = emojis.remove_emojis(token) # Elimina emojis.
        token = re.sub(r'\s+', ' ', token).strip() # Elimina espacios en blanco consecutivos.
        tokens.append(token)
        
    return " ".join(tokens)

<br>

Tras definir las funciones, procedemos a crear una nueva columumna en `df_names` con el título procesado.

In [9]:
tqdm.pandas()

df_names['tags'] = df_names['name'].progress_apply(clean_name)

100%|██████████████████████████████████████████████████| 1010000/1010000 [01:44<00:00, 9663.26it/s]


<br>

Como último paso, vamos a crear un diccionario con el que realizaremos los cambios de emojis por las palabras con las que aparece con mayor frecuencia. Para ello, nos apoyaremos en la función `create_emojis_dict`, la cual se encargará de crearlo y nos dará opción de coger las *n* primeras palabras que aparecen con mayor frecuencia. También tendremos la opción de excluir determinadas palabras que consideremos que no aportan demasiada información.

In [10]:
def create_emojis_dict(df, removable_words=[], num_tags=5):
    emojis_set = set()
    emoji_dict = dict()
    
    for emoji_title in df[df['emojis'] != '']['emojis'].to_list(): 
        emojis_set.update(emoji_title)
        
    df_titles_emojis = df[df['emojis'] != '']

    for emoji_char in emojis_set:
        tags = " ".join(df_titles_emojis[df_titles_emojis['emojis'].str.contains(emoji_char)]['tags'].to_list())
        tags_list = [tag for tag in tags.split(' ') if len(tag) > 0]
        words_count_dict = defaultdict(int)

        for tag in tags_list:
            words_count_dict[tag] += 1

        words_count_dict = dict(sorted(words_count_dict.items(), key=lambda item: item[1], reverse=True))

        result = [k for k,v in words_count_dict.items() 
                  if k not in removable_words and
                  not(k.isnumeric())][:num_tags]
        if len(result) > 0:
            emoji_dict[emoji_char] = result
            
    return emoji_dict

In [11]:
emojis_dict = create_emojis_dict(df_names, removable_words= ["music", "playlist", "song"])

<br>

Llegados a este punto, ya podemos crear el conjunto de etiquetas o *tokens* que contiene cada playlist de nuestro conjunto.

<br>

---

<br>


<a id="section21"></a>
### <font color="#92002A">2.1 - Construcción de la matriz TF-IDF</font>

<br>

Para poder entrenar nuestro modelo de recomendación a partir de un conjunto de documentos de texto, en nuestro caso los títulos de las playlists, es necesario representar estos datos en una matriz bidimensional. En el modelo _Bag of Words_, cada documento se representa a partir del conjunto de palabras que aparecen en él. Como paso previo a la construcción de la matriz de datos, se elabora un vocabulario con la unión de todos los términos que aparecen en algún documento. A partir de éste, se construye una matriz en la que cada fila representa un documento, y cada columna (cada característica) corresponde a un término. 

En la versión más básica, cada posición ${(d,t)}$ de la matriz contiene el valor para la medida ___Term Frequency___ (frecuencia de términos), que se denota *tf*$_{d,t}$, y refleja al número de veces que aparece el término $t$ en el documento $d$. Por ejemplo, dados estos tres documentos:

* Documento 1: "El objetivo de esta práctica es explicar el procesamiento básico de texto libre",
* Documento 2: "Uno de los enfoques utilizados es la bolsa de palabras",
* Documento 3: "Procesamiento de texto mediante bolsa de palabras",

la medida _Term Frequency_  generaría esta matriz de datos:

<br>
$$
X = \left[ \begin{array}{c | c | c | c | c | c | c }
El  & objetivo & de & esta & \cdots & bolsa & palabras \\
2 & 1 & 2 & 1 & \cdots & 0 & 0\\
0 & 0 & 2 & 0 & \cdots & 1 & 1\\
0 & 0 & 2 & 0 & \cdots & 1 & 1\\
\end{array}  \right] \begin{array}{c}
\\
Documento \, 1 \\
Documento \, 2 \\
Documento \, 3 \\
\end{array}
$$
<br>

En la matriz, *tf*$_{0,1}=2$, porque la palabra de índice $0$, "El", aparece dos veces en el documento $1$. 

Existen algunas variantes de esta medida, de modo que la matriz de datos se puede formar con:

* *tf*$_{d,t} \in \{0,1\}$ - Es decir, si el término aparece o no en el documento.
* $\lg(1+tf_{d,t})$ - Escala logarítmica.
* *tf*$_{d,t}/max_j$ *tf*$_{d,j}$ - Escala en relación al término más frecuente en el documento.
<br>


La frecuencia de términos comunes como "el", "a", "de", etc, suele ser alta para la mayoría de los documentos. La métrica ___Inverse Document Frequency___ (frecuencia inversa en documentos) penaliza los términos en proporción a la frecuencia con que aparecen en el conjunto de documentos. La frequencia inversa de un término se formula como:

$$
idf_t = log\left(\frac{\# \,documentos}{\# \,documentos \, que \, incluyen \, la \, palabra \, t }\right)
$$

En el ejemplo anterior:

* $ idf_{el} = log\left(\frac{3}{1}\right) = 1.098 $
* $idf_{objetivo} = log\left(\frac{3}{1}\right) = 1.098$
* $idf_{de} = log\left(\frac{3}{3}\right) = 0$
* $idf_{esta} = log\left(\frac{3}{1}\right) = 1.098$
* $idf_{bolsa} = log\left(\frac{3}{2}\right) = 0.405$
* $idf_{palabras} = log\left(\frac{3}{2}\right) = 0.405$

<br>
Existen algunas formulaciones alternativas como:

$$
idf_t = log\left(\frac{1 + \# \,documentos}{1 + \# \,documentos \, que \, incluyen \, la \, palabra \, t }\right) + 1
$$

<br>
<br>

Por último, lo habitual es utilizar una combinación de estas métricas, denominada ___Term Frequency Inverse Document Frequency (tf-idf)___ , que se formula como:

$$
tf-idf_{d,t} = tf_{d,t} \times idf_t
$$
<br>

Dados los documentos anteriores, la matriz con los valores *tf-idf*$_{d,t}$ sería:

<br>

$$
X = \left[ \begin{array}{c | c | c | c | c | c | c }
El  & objetivo & de & esta & \cdots & bolsa & palabras \\
2.2 & 1.1 & 2.2 & 1.1 & \cdots & 0 & 0\\
0 & 0 & 2.2 & 0 & \cdots & 0.81 & 0.81\\
0 & 0 & 2.2 & 0 & \cdots & 0.81 & 0.81\\
\end{array}  \right] \begin{array}{c}
\\
Documento \, 1 \\
Documento \, 2 \\
Documento \, 3 \\
\end{array}
$$

<br>

Para crear la matriz *tf-idf*, vamos a emplear la clase [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Dicha función forma parte del módulo de [extracción de características de texto](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text) de la librería [*scikit-learn*](https://scikit-learn.org/).

Antes de crear dicha matriz, en vez de usar el *tokenizer* que emplea por defecto `TfidfVectorizer`, vamos a definir uno propio. Para ello, nos vamos a basar en la función `clean_name` que empleamos para *normalizar* los títulos de las playlist. Ésta nueva función, llamada `name_tokenizer`, va a realizar el mismo proceso que la anterior salvo por los siguientes cambios:
* Sustituye los emojis del título por los dos primeros términos que se encuentran en el diccionario que hemos creado (`emojis_dict`). En caso de que el emoji no se encuentre en dicho diccionario, procederemos a incorporarlo al texto.
* Retornara una lista de `strings` con los tokens obtenidos del título de la playlist.

In [12]:
def name_tokenizer(name):
    emoji_list = emojis.get_emojis_list(name)
    
    name = str(name).lower()
    name = replace_apostrophe(name)
    name = dot_remover(name)
    name = re.sub(r"[~_]", ' ', name)
    name = re.sub(r"[^\w\s\.]", ' ', name)
    
    tokens = list()
    for token in word_tokenize(name):
        token = emojis.remove_emojis(token)
        token = re.sub(r'\s+', ' ', token).strip()
        tokens.append(token)
    
    emojis_translations = set()
    for e in emoji_list:
        if e in emojis_dict:
            emojis_translations.update(emojis_dict[e][:2])
        else:
            emojis_translations.add(e)
            
    for t in emojis_translations:
        if t not in tokens:
            tokens.append(t)
    
    return tokens

In [13]:
# Obtenemos una lista con los títulos de las playlists
pls_names_list = df_names['name'].to_list()

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Definimos el vectorizador indicando que vamos a usar nuestro propio tokenizer
vectorizer = TfidfVectorizer(tokenizer=name_tokenizer)
"""
# Creamos la matriz tf_idf
X = vectorizer.fit_transform(pls_names_list)
print(f"Dimensiones: {X.shape}")
"""

'\n# Creamos la matriz tf_idf\nX = vectorizer.fit_transform(pls_names_list)\nprint(f"Dimensiones: {X.shape}")\n'

<br>

Una vez creada la matriz, creamos un diccionario que contendrá:
* El vectorizador creado
* La lista que conforman las características obtenidas.
* La matriz *tf-idf*.

In [15]:
tfidf_data_dict = dict()
"""
tfidf_data_dict['vectorizer'] = vectorizer
tfidf_data_dict['features_list'] = vectorizer.get_feature_names()
tfidf_data_dict['matrix'] = X
"""

"\ntfidf_data_dict['vectorizer'] = vectorizer\ntfidf_data_dict['features_list'] = vectorizer.get_feature_names()\ntfidf_data_dict['matrix'] = X\n"

In [16]:
import pickle
"""
# Guardamos el diccionario creado en un fichero pickle
with open(TFIDF_DATA_FILE, "wb") as write_file:
    pickle.dump(tfidf_data_dict, write_file, protocol=pickle.HIGHEST_PROTOCOL)
"""

'\n# Guardamos el diccionario creado en un fichero pickle\nwith open(TFIDF_DATA_FILE, "wb") as write_file:\n    pickle.dump(tfidf_data_dict, write_file, protocol=pickle.HIGHEST_PROTOCOL)\n'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section3"></a>
## <font color="#92002A">3 - Obtención de géneros musicales (artistas)</font>
<br>

En esta sección, vamos a obtener los géneros musicales de los artistas que conforman nuestro conjunto de datos (puesto que *Spotify* no proporciona los géneros para pistas y/o álbumes). Usaremos esta información para etiquetar las pistas que pertenezcan a un artista con los mismos géneros musicales.

<br>

Creamos el gestor de *Spotipy* con el cual obtendremos la información de *Spotify*, empleando los credenciales necesarios para acceder a la API:

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [18]:
sp_client_id = '78007ce51a07476f8edc56984f02b9f3' #SpotifyClientID
sp_client_secret = 'f7d87b63601946b1b1d0d5ff8b2524e0' #SpotifyClientSecret

# Crea el gestor
client_credentials_manager = SpotifyClientCredentials(client_id=sp_client_id, client_secret=sp_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

<br>

Como en cada llamada a la API de *Spotify* sólo podemos obtener la información de 50 artistas, dividimos la lista que contiene los identificadores de los artistas en bloques de 50 identificadores.

In [19]:
def get_list_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

<br>

Descargamos la información de los artistas:

In [20]:
# Lista con los identificadores de los artistas
artists_ids = pd.read_csv('MPD_CSV/mpd.artists.csv', index_col=0)['artist_id'].to_list()

In [23]:
import time

artist_chunks = get_list_chunks(artists_ids, 50)
artists_results = []

"""
for chunk in tqdm(artist_chunks, total=len(artists_ids)/50):
    results = sp.artists(chunk)['artists']
    artists_results += [{'artist_id': d['id'], 'genres' : d['genres']} for d in results
                        if d != None and 'genres' in d.keys()]
    time.sleep(0.15)
"""

with open("backup/artists_genres.pkl", "rb") as read_file:
    artists_results = pickle.load(read_file)

<br>

Una vez obtenida la información de los artistas, creamos un diccionario cuyos elementos tendran como clave el identificador del artista y como valor el conjunto de generos al que pertenece, separados por el caracter `|`:

In [26]:
artist_genres_dict = dict()
for artist in artists_results:
    artist_genres_dict[artist['artist_id']] = "|".join(artist['genres'])

<br>

Creamos un *dataframe* de *pandas* a partir del diccionario obtenido:

In [29]:
df_genres = pd.DataFrame.from_dict(artist_genres_dict, orient='index', columns=['genres'])

<br>

Nos quedamos con aquellos artistas de los cuales se disponga de información sobre sus generos y almacenmos la información en un fichero *CSV*:

In [33]:
df_genres = df_genres[df_genres['genres'] != '']
df_genres.to_csv('MPD_CSV/mpd.artists-genres.csv')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section4"></a>
## <font color="#92002A">4 - Obtención de fechas de lanzamiento (álbumes)</font>

<br>

Habitualmente, se suelen crear playlists que contienen pistas que pertenecen a un año, década o periodo de tiempo determinado, recopilando lo mejor de ese tiempo, canciones favoritas de una época, etc.

Como *Spotify* proporciona la fecha de lanzamiento de los álbumes que posee, vamos a obtener dicha información para el conjunto de álbumes que componen nuestro conjunto de datos:

In [35]:
# Lista con los identificadores de los álbumes
album_ids = pd.read_csv('MPD_CSV/mpd.albums.csv', index_col=0)['album_id'].to_list()

<br>

Al igual que sucede con los artistas, por cada llamada a la API de *Spotify*, sólo podemos obtener la información de 20 álbumes. Para ello volvemos a crear grupos de álbumes para realizar las llamadas a la API y obtener la fecha de lanzamiento:

In [45]:
album_chunks = get_list_chunks(album_ids, 20)
album_results = []

"""
for chunk in tqdm_nb(album_chunks, total=len(album_ids)/20):
    results = sp.albums(chunk)['albums']
    album_results += [{'id': d['album_id'], 'release_date' : d['release_date']} for d in results
                      if d != None and 'release_date' in d.keys()]
    time.sleep(0.15)
"""

with open("backup/albums_release.pkl", "rb") as read_file:
    album_results = pickle.load(read_file)

<br>

Tras terminar de descargarse la información de los álbumes, creamos un *dataframe* de *pandas* a partir de la lista de diccionarios obtenida:

In [51]:
df_albums_release = pd.DataFrame(album_results)
df_albums_release.set_index('album_id', inplace=True)

<br>

Eliminamos aquellos álbumes que no dispongan de fecha de lanzamiento, en el caso de existir, y guardamos la información en un fichero *CSV*:

In [53]:
df_albums_release = df_albums_release[df_albums_release['release_date'] != '']
df_albums_release.to_csv('MPD_CSV/mpd.albums-releasedate.csv')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section5"></a>
## <font color="#92002A">5 - Obtención de características músicales (pistas)</font>
<br>

Otro tipo de información que nos ofrece *Spotify*, y que puede resultarnos útil para crear nuestro modelo de recomendación, son las características de audio (o *audio features*) para las pistas, obtenidas tras realizar un análisis de audio de la canción. Las características disponibles son las siguientes:

* **Capacidad de baile (*Danceability*)**: La capacidad de baile describe cómo de adecuada es una pista para bailar en función de una combinación de elementos musicales que incluyen el tempo, la estabilidad del ritmo, la fuerza del ritmo y la regularidad general. Un valor de 0.0 es menos bailable y 1.0 es el más bailable. 

* **Acústica (*Acousticness*)**: Valor comprendido entre 0.0 y 1.0 que indica el nivel de acústica de la pista. 

* **Energía (*Energy*)**: Valor comprendido entre 0.0 a 1.0 y que representa una medida de percepción de intensidad y actividad. Por lo general, las pistas enérgicas se sienten rápidas, de volumen alto y ruidosas. 

* **Instrumentalidad (Instrumentalness)**: Predice si una pista no contiene voces. Cuanto más cercano esté el valor de instrumentalidad a 1.0, mayor será la probabilidad de que la pista no contenga contenido vocal.

* **Vivacidad (*Liveness*)**: Detecta la presencia de una audiencia en la grabación. Los valores de vivacidad más altos representan una mayor probabilidad de que la pista se haya interpretado en vivo.

* **Sonoridad (*Loudness*)**: Indica la sonoridad general de una pista en decibelios (dB). Los valores de sonoridad se promedian en toda la pista. Los valores típicos oscilan entre -60 y 0 db.

* **Habla (Speechiness)**: El habla detecta la presencia de palabras habladas en una pista. Cuanto más exclusivamente parecida a un discurso sea la grabación (por ejemplo, programa de entrevistas, audiolibro, poesía), más cercano a 1.0 será el valor del atributo.

* **Tempo**: El tempo total estimado de una pista en pulsaciones por minuto (o *BPM*). En terminología musical, el tempo es la velocidad o el ritmo de una pieza determinada y se deriva directamente de la duración media del tiempo. 

* **Valencia (*Valence*)**: Medida entre 0.0 y 1.0 que describe la positividad musical que transmite una pista. Las pistas con valencia alta suenan más positivas (feliz, alegre, eufórico, ...), mientras que las pistas con valencia baja suenan más negativas (triste, deprimido, enojado, ...).

<br>

In [54]:
# Lista con los identificadores de las pistas
track_ids = pd.read_csv('MPD_CSV/mpd.tracks.csv', index_col=0)['track_id'].to_list()

<br>

Como ha sucedido en los casos anteriores, por cada llamada a la API de *Spotify*, sólo podemos obtener la información de 100 pistas. Para ello volvemos a crear grupos de pistas para realizar las llamadas a la API y obtener las características musicales:

In [56]:
track_chunks = get_list_chunks(track_ids, 100)
track_results = []

"""
for chunk in tqdm(track_chunks, total=len(track_ids)/100):
    track_results += sp.audio_features(tracks=chunk)
    time.sleep(0.15)
"""

with open("backup/audiofeats_full.pkl", "rb") as read_file:
    track_results = pickle.load(read_file)

<br>

Una vez completada la descarga de las características musicales, cargamos la información obtenida en un *dataframe* de *pandas*:

In [67]:
df_audiofeats = pd.DataFrame(track_results)

<br>

Si estudiamos el contenido del *dataframe*, podemos ver que también se han obtenido algunos valores (como la url de la pista, duración, duración, ...) que no necesitamos:

In [68]:
df_audiofeats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5048058 entries, 0 to 5048057
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   danceability      float64
 1   energy            float64
 2   key               int64  
 3   loudness          float64
 4   mode              int64  
 5   speechiness       float64
 6   acousticness      float64
 7   instrumentalness  float64
 8   liveness          float64
 9   valence           float64
 10  tempo             float64
 11  type              object 
 12  id                object 
 13  uri               object 
 14  track_href        object 
 15  analysis_url      object 
 16  duration_ms       int64  
 17  time_signature    int64  
dtypes: float64(9), int64(4), object(5)
memory usage: 693.2+ MB


<br>

Eliminamos dichos valores, establecemos el id de la pista como índice y lo guardamos en un fichero de formato *CSV*:

In [69]:
df_audiofeats.drop(columns=['mode','type','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)
df_audiofeats.set_index('id', inplace=True)
df_audiofeats.index.name = 'track_id'

df_audiofeats.to_csv('MPD_CSV/mpd.tracks-audiofeats.csv')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>